In [1]:
import yfinance as yf
import keras.backend as K
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler ,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score, mean_squared_error,mean_absolute_percentage_error
import tensorflow as tf
from tensorflow import keras
from keras.layers import LSTM,Dropout, BatchNormalization, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from keras.models import Sequential
from keras.optimizer_v2.adam import Adam
from keras.optimizer_v2.rmsprop import RMSprop
import talib

In [2]:
# Get the data for the SPY ETF by specifying the stock ticker, start date, and end date
data = yf.download('0005.hk',"2017-09-28","2021-09-24")
# data.drop("Adj Close",axis=1,inplace=True)
data.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-09-28,76.400002,77.099998,76.050003,76.199997,64.386978,22520013
2017-09-29,76.500000,76.900002,76.300003,76.550003,64.682732,16766252
2017-10-03,76.800003,77.449997,76.699997,77.449997,65.443199,24369212
2017-10-04,77.500000,77.949997,77.500000,77.849998,65.781197,21197563
2017-10-06,77.599998,77.800003,77.199997,77.349998,65.358711,23715110


In [3]:
data.shift(1).head(90)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-09-28,NaN,NaN,NaN,NaN,NaN,NaN
2017-09-29,76.400002,77.099998,76.050003,76.199997,64.386978,22520013.0
2017-10-03,76.500000,76.900002,76.300003,76.550003,64.682732,16766252.0
2017-10-04,76.800003,77.449997,76.699997,77.449997,65.443199,24369212.0
2017-10-06,77.500000,77.949997,77.500000,77.849998,65.781197,21197563.0
...,...,...,...,...,...,...
2018-02-01,83.449997,84.500000,83.000000,84.199997,71.864357,36516373.0
2018-02-02,83.500000,84.150002,83.449997,84.050003,71.736343,33272295.0
2018-02-05,83.550003,84.000000,83.250000,83.699997,71.437614,21777829.0


In [4]:
data['H-L'] = data['High'] - data['Low']
data['O-C'] = data['Close'] - data['Open']
# data=pd.merge(data,data["Close"].pct_change(),left_index=True,right_index=True)
data["% Change"]=data["Close"].shift(1).pct_change()
data['3day MA'] = data['Close'].shift(1).rolling(window = 3).mean()
data['10day MA'] = data['Close'].shift(1).rolling(window = 10).mean()
data['30day MA'] = data['Close'].shift(1).rolling(window = 30).mean()
data['Std_dev']= data['Close'].shift(1).rolling(5).std()
data['Money Flow']=talib.MFI(data['High'], data['Low'], data['Close'], data['Volume'], timeperiod=14)
# data['RSI'] = talib.RSI(data['Close'].values, timeperiod = 9)
# data['Williams %R'] = talib.WILLR(data['High'].values, data['Low'].values, data['Close'].values, 7)
data.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'H-L', 'O-C',
       '% Change', '3day MA', '10day MA', '30day MA', 'Std_dev', 'Money Flow'],
      dtype='object')

In [5]:
data.dropna(inplace=True)
data.isnull().sum()

Open          0
High          0
Low           0
Close         0
Adj Close     0
Volume        0
H-L           0
O-C           0
% Change      0
3day MA       0
10day MA      0
30day MA      0
Std_dev       0
Money Flow    0
dtype: int64

In [11]:
from sklearn.feature_selection import VarianceThreshold
X=data.drop('Adj Close',axis=1)
Y=data['Close']
var = VarianceThreshold(threshold=0.3)
var = var.fit(X,Y)
X

,Open,High,Low,Close,Volume,H-L,O-C,% Change,3day MA,10day MA,30day MA,Std_dev,Money Flow
Date,,,,,,,,,,,,,
2017-11-13,75.900002,76.250000,75.599998,75.650002,12740554,0.650002,-0.250000,0.001318,75.716665,75.939999,76.705000,0.302075,49.736006
2017-11-14,75.199997,75.449997,75.050003,75.150002,16683838,0.399994,-0.049995,-0.003950,75.816666,75.820000,76.686666,0.233452,44.898521
2017-11-15,75.150002,75.300003,75.050003,75.099998,10045036,0.250000,-0.050003,-0.006609,75.583333,75.715000,76.640000,0.336153,40.847101
2017-11-16,75.500000,75.699997,75.250000,75.349998,8504207,0.449997,-0.150002,-0.000665,75.300001,75.560000,76.561666,0.394334,38.169113
2017-11-17,75.349998,75.650002,75.099998,75.250000,12812063,0.550003,-0.099998,0.003329,75.199999,75.510000,76.478333,0.357770,27.800601
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-16,40.599998,40.599998,39.599998,40.000000,19553563,1.000000,-0.599998,-0.002488,40.333333,40.795000,42.193333,0.336156,27.480926
2021-09-17,39.700001,40.400002,39.099998,40.250000,28266370,1.300003,0.549999,-0.002494,40.100000,40.660001,42.083333,0.396233,25.640724
2021-09-20,39.900002,39.950001,38.950001,39.000000,23796781,1.000000,-0.900002,0.006250,40.116666,40.540001,41.985000,0.269259,25.303577


In [7]:
cols = var.get_support(indices=True)
cols

array([ 0,  1,  2,  3,  4,  8,  9, 10, 12])

In [8]:
features = X.columns[cols]
features

Index(['Open', 'High', 'Low', 'Close', 'Volume', '3day MA', '10day MA',
       '30day MA', 'Money Flow'],
      dtype='object')

In [9]:
cor = data.corr()
# Consider correlations only with the target variable
cor_target = abs(cor['Close'])

#Select correlations with a correlation above a threshold
features = cor_target[cor_target>0.1]
features

Open         0.999371
High         0.999710
Low          0.999703
Close        1.000000
Adj Close    0.991476
Volume       0.290776
3day MA      0.997963
10day MA     0.994940
30day MA     0.984986
Name: Close, dtype: float64

In [ ]:
data_options = pd.read_csv("Put_Call_Ratio.csv",skiprows=1)
data_options.columns

In [ ]:
data_options['Date (D/M/Y)']=pd.to_datetime(data_options['Date (D/M/Y)'], format="%d/%m/%Y")
data_options.set_index('Date (D/M/Y)',inplace=True)
data_options.sort_index(ascending=True,inplace=True)
data_options.columns

In [ ]:
# data=pd.merge(data,data_options[" Put/Call Ratio"], left_index=True,right_index=True)
data=data
# data.drop(["Volume","Adj Close"], axis=1,inplace=True)

In [ ]:
data.dropna(inplace=True)
data.isnull().sum()

In [ ]:
data.head()

In [ ]:
data_price=data["Close"]
train=data.shift(1).dropna().values
sc = MinMaxScaler(feature_range=(0,1))
train=sc.fit_transform(train)
def processData(data,data_price,lb):
    X,Y = [],[]
    for i in range(lb-1,len(data)-lb-1):
        X.append(data[i-(lb-1):i])
        Y.append(data_price[(i)])
    return np.array(X),np.array(Y)

lb=7
X,y = processData(train,data_price,lb)
X_train,X_test = X[:int(X.shape[0]*0.90)],X[int(X.shape[0]*0.90):]
y_train,y_test = y[:int(y.shape[0]*0.90)].reshape(-1,1),y[int(y.shape[0]*0.90):].reshape(-1,1)
# print(type(X_train))
print(X[0])
print(y[0])
print(data.shape)

In [ ]:
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

In [ ]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape= (X_train.shape[1], X_train.shape[2])))
# model.add(Dropout(0.7))
model.add(LSTM(64, return_sequences=False))
# model.add(Dropout(0.5))
model.add(Dense(32))
model.add(Dense(1))
model.compile(
  loss="mean_squared_error",
  optimizer='Adam'
)

In [ ]:
model.summary() 

In [ ]:
#Fit model with history to check for overfitting
# es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=40, min_delta=0.0001)
history = model.fit(X_train,y_train,epochs=300,batch_size=20,validation_data=(X_test,y_test),shuffle=False)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()

In [ ]:
plt.plot(np.arange(0, len(y_train)), y_train, 'g', label="history")
plt.plot(np.arange(len(y_train), len(y_train) + len(y_test)), y_test, marker='.', label="true")
plt.plot(np.arange(len(y_train), len(y_train) + len(y_test)), y_pred, 'r', label="prediction")
plt.ylabel('Value')
plt.xlabel('Time Step')
plt.legend()
plt.show();

In [ ]:
# plot_x=data[int(X.shape[0]*0.90):]
# plot_x["Prediction"]=y_test

In [ ]:
plt.plot(y_test, marker='.', label="true")
plt.plot(y_pred, 'r',marker='*', label="prediction")
plt.ylabel('Value')
plt.xlabel('Time Step')
plt.legend()
plt.show();

In [ ]:
mean_absolute_percentage_error(y_test,y_pred)*100

In [ ]:
mean_squared_error(y_test,y_pred)

In [ ]:
y_pred_lstm_final = np.zeros((len(y_pred)-1,))
y_test_t_final = np.zeros((len(y_test)-1,))
for i in range(len(y_pred)-1):
    if y_pred[i+1] >= y_pred[i]:
        y_pred_lstm_final[i] = 1
    else:
        y_pred_lstm_final[i] = 0
        
#convert prediction into binary output (up or down movement)
for i in range(len(y_test)-1):
    if y_test[i+1] >= y_test[i]:
        y_test_t_final[i] = 1
    else:
        y_test_t_final[i] = 0

print(y_pred_lstm_final[0:15])
print(y_test_t_final[0:15])
print("Predicted direction accuracy (Up/down):")
print(accuracy_score(y_test_t_final,y_pred_lstm_final))